# Genetic Algorithm for Timetabling Problem

## 0 - Configurations


## 0.1 - Importing Libraries

In [1]:
from dataclasses import dataclass, Field
from typing import List, Dict, Set, Tuple, Any
from datetime import datetime
import time
import random
import json

## 0.2 - Environmental Variables

In [2]:
MOCK_DATA_FOLDER_PATH = "../../../data/mock/"

INPUT_FILE_PATH = MOCK_DATA_FOLDER_PATH + "input.json"

## 0.3 - Custom Functions

In [3]:
def load_data(file_path, dataclasses_relationships) -> dict:
    """
    Load data from a json file and return a dictionary with the data loaded as dataclasses of each entity
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
        timetabling_data = {}
        for entity, values in data.items():
            print(f"Loading {entity} data")
            timetabling_data[entity] = []
            for value in values:
                entity_value = dataclasses_relationships[entity](value)
                timetabling_data[entity].append(entity_value)
            print(f"{len(timetabling_data[entity])} of {entity} loaded")
        return timetabling_data

# 1 - Model

## 1.1 - Entities of the Problem

In [4]:
@dataclass
class Time:
    id: int
    start: str
    end: str

    def __init__(self, time_data: dict) -> None:
        self.id = time_data["id"]
        self.start = time_data["start"]
        self.end = time_data["end"]

In [5]:
@dataclass
class Day:
    id: int
    name: str

    def __init__(self, day_data: dict) -> None:
        self.id = day_data["id"]
        self.name = day_data["name"]

In [6]:
@dataclass
class Availability:
    day_id: int
    time_id: int

In [7]:
@dataclass
class ClassGroup:
    id: int
    students_amount: int

    def __init__(self, class_group_data: dict) -> None:
        self.id = class_group_data["id"]
        self.students_amount = class_group_data["studentsAmount"]

In [8]:
@dataclass
class Classroom:
    id: int
    capacity: int
    available_days_ids: List[int]
    available_time_ids: List[int]
    given_availability: Availability|None = None
    
    def __init__(self, classroom_data: dict) -> None:
        self.id = classroom_data["id"]
        self.capacity = classroom_data["capacity"]
        self.available_days_ids = classroom_data["days"]
        self.available_time_ids = classroom_data["times"]
        
    @property
    def availabilities(self) -> List[Availability]:
        """
        Returns a list of all possible availabilities for the professor
        """
        availabilities = []
        for day_id in self.available_days_ids:
            for time_id in self.available_time_ids:
                availabilities.append(Availability(day_id, time_id))
        return availabilities
    
    def set_random_availability(self) -> None:
        """
        Sets a random availability for the professor
        """
        self.given_availability = random.choice(self.availabilities)

In [9]:
@dataclass
class Subject:
    id: int
    course_load: int

    def __init__(self, subject_data: dict) -> None:
        self.id = subject_data["id"]
        self.course_load = subject_data["hours"]

In [10]:
@dataclass
class Professor:
    id: int
    available_days_ids: List[int]
    available_time_ids: List[int]
    taughtble_subjects_ids: List[int]
    given_subject_id: int|None = None
    given_availability: Availability|None = None

    def __init__(self, professor_data: dict) -> None:
        self.id = professor_data["id"]
        self.available_days_ids = professor_data["days"]
        self.available_time_ids = professor_data["times"]
        self.taughtble_subjects_ids = professor_data["subjects"]

    @property
    def availabilities(self) -> List[Availability]:
        """
        Returns a list of all possible availabilities for the professor
        """
        availabilities = []
        for day_id in self.available_days_ids:
            for time_id in self.available_time_ids:
                availabilities.append(Availability(day_id, time_id))
        return availabilities
    
    def set_random_availability(self) -> None:
        """
        Sets a random availability for the professor
        """
        self.given_availability = random.choice(self.availabilities)

    def set_random_subject(self) -> None:
        """
        Sets a random subject for the professor
        """
        self.given_subject_id = random.choice(self.taughtble_subjects_ids)

In [11]:
@dataclass
class Gene:
    """
    A gene is a combination of a class group, a classroom, a subject and a professor (a possible timeslot entry in the timetable)
    """
    class_group: ClassGroup
    classroom: Classroom
    subject: Subject
    professor: Professor

    def __init__(self) -> None:
        pass
    
    def create_gene(self, class_group: ClassGroup, classroom: Classroom, subject: Subject, professor: Professor) -> None:
        """
        Create a gene with the **given parameters** and randomly select the availability for the classroom and the professor
        """
        self.class_group = class_group
        classroom.set_random_availability()
        self.classroom = classroom
        self.subject = subject
        professor.set_random_availability()
        self.professor = professor

    def generate_random_gene(self, class_groups: List[ClassGroup], classrooms: List[Classroom], subjects: List[Subject], professors: List[Professor]) -> None:
        """
        Create a gene with complete random parameters and availability for the classroom and the professor
        """
        self.class_group = random.choice(class_groups)
        random_classroom_availability = random.choice(classrooms)
        random_classroom_availability.set_random_availability()
        self.classroom = random_classroom_availability
        self.subject = random.choice(subjects)
        random_professor_availability = random.choice(professors)
        random_professor_availability.set_random_availability()
        random_professor_availability.set_random_subject()
        self.professor = random_professor_availability

    @property
    def __is_valid(self) -> bool:
        """
        Gene is valid if the classroom capacity is greater or equal than the class group students amount
        """
        return self.class_group.students_amount <= self.classroom.capacity
    
    @property
    def __is_feasible(self) -> bool:
        """
        Gene is feasible if the classroom and the professor have the same availability
        """
        return self.classroom.given_availability == self.professor.given_availability
    
    @property
    def amount_of_internal_conflicts(self) -> int:
        """
        Returns the amount of conflicts in the gene; from 0 to 2; Evaluates if the gene is valid and feasible
        """
        conflicts = 0
        if not self.__is_valid:
            conflicts += 1
        if not self.__is_feasible:
            conflicts += 1
        #print(f"Gene {self} has {conflicts} conflicts and is valid: {self.__is_valid} and feasible: {self.__is_feasible}")
        return conflicts

In [12]:
@dataclass
class Chromosome:
    """
    A chromosome is a combination of genes that can represent a possible solution for the problem (the complete timetable)
    """
    genes: List[Gene]

    def __init__(self) -> None:
        self.genes = []

    def add_gene(self, gene: Gene) -> None:
        self.genes.append(gene)

    @property
    def amount_of_conflicts_between_genes(self) -> int:
        """
        Returns the amount of conflicts between the genes in the chromosome; For each pair of genes, evaluates if the classroom and the professor have the same availability; Therefore evaluating if the genes are feasible or if they occupy the same timeslot in the timetable.
        """
        conflicts_between_genes = 0
        # For each pair of genes
        for i in range(len(self.genes)):
            for j in range(i + 1, len(self.genes)):
                # Evaluate if the classroom are the same and have the same availability
                if self.genes[i].classroom == self.genes[j].classroom and self.genes[i].classroom.given_availability == self.genes[j].classroom.given_availability:
                    conflicts_between_genes += 1
                # Evaluate if the professor are the same and have the same availability
                if self.genes[i].professor == self.genes[j].professor and self.genes[i].professor.given_availability == self.genes[j].professor.given_availability:
                    conflicts_between_genes += 1
        #print(f"Chromosome with total amount of {conflicts_between_genes} conflicts between genes")
        return conflicts_between_genes
        
    @property
    def amount_of_conflicts(self) -> int:
        """
        Returns the amount of conflicts in the chromosome; Evaluates the amount of internal conflicts in each gene and the amount of conflicts between the genes
        """
        conflicts = 0
        for gene in self.genes:
            conflicts += gene.amount_of_internal_conflicts
        conflicts += self.amount_of_conflicts_between_genes
        #print(f"Chromosome with total amount of {conflicts} conflicts")
        return conflicts
    
    @property
    def avg_amount_of_conflicts(self) -> float:
        avg_amount_of_conflicts = self.amount_of_conflicts / len(self.genes)
        #print(f"Chromosome with average amount of {avg_amount_of_conflicts} conflicts")
        return avg_amount_of_conflicts

    @property
    def elite_fitness(self) -> float:
        fitness = round(1 - self.amount_of_conflicts * 0.1, 2)
        if fitness < 0:
            #print(f"Chromosome with negative fitness, floored to 0 !")
            fitness = 0
        return fitness

In [13]:
@dataclass
class Population:
    """
    A population is a collection of chromosomes that can represent a set of possible solutions for the problem
    """
    chromosomes: List[Chromosome]

    def __init__(self) -> None:
        self.chromosomes = []

    def add_chromosome(self, chromosome: Chromosome) -> None:
        self.chromosomes.append(chromosome)

    def generate_population_from_data(self, population_size: int, class_groups: List[ClassGroup], classrooms: List[Classroom], subjects: List[Subject], professors: List[Professor], course_load_per_month: int = 36) -> None:
        """
        Generate a initial population of chromosomes (timetables) with genes (timeslots) with the combinatorial selection of class groups, classrooms, subjects and professors and the random selection of availabilities (day and time) for the classroom and the professor
        """
        print(f"Generating population of size {population_size} with {len(class_groups)} class groups, {len(classrooms)} classrooms, {len(subjects)} subjects and {len(professors)} professors and a course load threshold of {course_load_per_month} hours")
        for _ in range(population_size):
            chromosome: Chromosome = Chromosome()
            for class_group in class_groups:
                for classroom in classrooms:
                    for professor in professors:
                        for taughtble_subject_id in professor.taughtble_subjects_ids:
                            for subject in subjects:
                                if subject.id == taughtble_subject_id:
                                    for class_slot in range(subject.course_load // course_load_per_month):
                                        gene: Gene = Gene()
                                        gene.create_gene(class_group, classroom, subject, professor)
                                        chromosome.add_gene(gene)
            self.add_chromosome(chromosome)
        print(f"Population generated with {len(self.chromosomes)} chromosomes")

    @property
    def best_chromosome(self) -> Chromosome:
        best_chromosome = max(self.chromosomes, key=lambda chromosome: chromosome.elite_fitness)
        #print(f"Best chromosome with fitness of {best_chromosome.elite_fitness}")
        return best_chromosome
    
    def order_population_by_fitness(self, reverse: bool = False) -> None:
        """
        Order the chromosomes by fitness and select the best individuals **(elitism)**
        """
        self.chromosomes.sort(key=lambda chromosome: chromosome.elite_fitness, reverse=reverse)


## 1.2 - Custom Metrics Evaluator

In [14]:
@dataclass
class IterationEvaluationMetrics:
    """
    A set of partial indicators about a iteration of the genetic algorithm;
    With those indicators, it is possible to analyze the performance of the model through execution time and the quality of the solutions;
    * Conflicts are the hard constraints that are not satisfied in the timetable;
    * Elite fitness is a metric that represents the quality of the best solution found in the population given some fitness function;
    """
    iteration: int
    chromosome: Any
    avg_conflicts: float
    elite_fitness: float
    time_elapsed: float

In [15]:
@dataclass
class EvaluationMetrics:
    """
    A data structure to store the evaluation metrics of genetic algorithms models;
    It tracks partial indicators about each iteration and the best indicators of the model;
    """
    iteration_history: List[IterationEvaluationMetrics]
    best_iteration: IterationEvaluationMetrics
    total_time_elapsed: float

    def __init__(self, main_metric_of_evaluation: str) -> None:
        if main_metric_of_evaluation not in ["conflicts", "elite_fitness"]:
            raise ValueError("The main metric of evaluation must be 'conflicts' or 'elite_fitness'")
        self.main_metric_of_evaluation = main_metric_of_evaluation
        self.iteration_history = []
        self.best_iteration = IterationEvaluationMetrics(0, None, 0, 0, 0)

    def __compare_conflicts(self, iteration_evaluation_metrics: IterationEvaluationMetrics) -> None:
        """
        Compare the conflicts of the iteration with the best iteration found so far;
        """
        if iteration_evaluation_metrics.avg_conflicts < self.best_iteration.avg_conflicts:
            print(f"New best iteration found with {iteration_evaluation_metrics.avg_conflicts} conflicts !")
            self.best_iteration = iteration_evaluation_metrics

    def __compare_elite_fitness(self, iteration_evaluation_metrics: IterationEvaluationMetrics) -> None:
        """
        Compare the elite fitness of the iteration with the best iteration found so far;
        """
        if iteration_evaluation_metrics.elite_fitness > self.best_iteration.elite_fitness:
            print(f"New best iteration found with {iteration_evaluation_metrics.elite_fitness} elite fitness !")
            self.best_iteration = iteration_evaluation_metrics

    def add_iteration_evaluation_metrics(self, iteration_evaluation_metrics: IterationEvaluationMetrics) -> None:
        """
        Add a new iteration evaluation metrics to the evaluation metrics data structure;
        The main metric to compare the quality of the solution is the average of conflicts in the timetable (the N number of hard constraints not satisfied);
        """
        print(f"Iteration {iteration_evaluation_metrics.iteration} - Conflicts: {iteration_evaluation_metrics.avg_conflicts} - Elite Fitness: {iteration_evaluation_metrics.elite_fitness} - Time Elapsed: {iteration_evaluation_metrics.time_elapsed}")
        self.iteration_history.append(iteration_evaluation_metrics)
        if self.main_metric_of_evaluation == "conflicts":
            self.__compare_conflicts(iteration_evaluation_metrics)
        elif self.main_metric_of_evaluation == "elite_fitness":
            self.__compare_elite_fitness(iteration_evaluation_metrics)

    def calculate_total_time_elapsed(self, start_time: float) -> None:
        """
        Calculate the total time elapsed in the execution of the genetic algorithm model;
        """
        self.total_time_elapsed = time.time() - start_time

In [16]:
class MetricsEvaluator:
    """
    A class to manage the workflow of the evaluation metrics of genetic algorithms models;
    It tracks the performance of the model through the evaluation of the quality of the solutions and the execution time;
    """
    def __init__(self, model: str, main_metric_of_evaluation: str) -> None:
        print(f"Creating MetricsEvaluator of Model: {model}, starting evaluation")
        self.model: str = model
        self.metrics: EvaluationMetrics = EvaluationMetrics(main_metric_of_evaluation)
        self.start_time = time.time()

    def __repr__(self) -> str:
        return f"MetricsEvaluator({self.model})"
    
    def __str__(self) -> str:
        return f"MetricsEvaluator of Model: {self.model} with Metrics: {self.metrics}"

    def start_iteration(self) -> None:
        """
        Start the timer of the iteration to evaluate the execution time of the genetic
        """
        self.iteration_start_time: float = time.time()
        print(f"Starting iteration {len(self.metrics.iteration_history) + 1}")

    def finish_iteration(self, iteration: int, chromosome: Any, avg_conflicts: float, elite_fitness: float) -> None:
        """
        Finish the iteration, save it's partial metrics and calculate the time elapsed
        """
        iteration_time_elapsed: float = time.time() - self.iteration_start_time
        self.metrics.add_iteration_evaluation_metrics(IterationEvaluationMetrics(iteration, chromosome, avg_conflicts, elite_fitness, iteration_time_elapsed))
        print(f"Iteration {iteration} finished")

    def __save_evaluation(self) -> None:
        """
        Save the evaluation metrics of the genetic algorithm model in a json file
        """
        print(f"Saving evaluation metrics of the model {self.model}")
        with open(f"./logs/{self.model}_evaluation_metrics_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json", 'w') as file:
            json.dump(self.metrics, file, default=lambda x: x.__dict__)
        print(f"Metrics of the model saved")

    def finish_evaluation(self) -> None:
        """
        Finish the evaluation of the genetic algorithm model therefore calculating the total time elapsed and saving the evaluation metrics
        """
        self.metrics.calculate_total_time_elapsed(self.start_time)
        print(f"With a total time elapsed of {self.metrics.total_time_elapsed} seconds and {len(self.metrics.iteration_history)} iterations, the best iteration found was {self.metrics.best_iteration} which is the best solution found by the model with {self.metrics.best_iteration.avg_conflicts} conflicts and {self.metrics.best_iteration.elite_fitness} elite fitness")
        self.__save_evaluation()
    

## 1.3 Genetic Algorithm

In [17]:
class GeneticAlgorithm:
    def __init__(self, population_size: int, mutation_rate: float, crossover_rate: float, generations: int, fitness_threshold: float, course_load_one_class_threshold: int, class_groups: List[ClassGroup], classrooms: List[Classroom], subjects: List[Subject], professors: List[Professor]):
        self.metrics_evaluator = MetricsEvaluator("Genetic Algorithm", "elite_fitness")
        population = Population()
        population.generate_population_from_data(population_size, class_groups, classrooms, subjects, professors, course_load_one_class_threshold)
        self.population = population
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.generations = generations
        self.fitness_threshold = fitness_threshold
        self.course_load_one_class_threshold = course_load_one_class_threshold
        self.class_groups = class_groups
        self.classrooms = classrooms
        self.subjects = subjects
        self.professors = professors

    @staticmethod
    def __selection(population: Population) -> Population:
        """
        Select the best individuals of the population to the matching pool
        """
        population.order_population_by_fitness()
        matching_pool: Population = Population()
        for i in range(len(population.chromosomes)):
            if random.random() < (i / len(population.chromosomes)):
                matching_pool.add_chromosome(population.chromosomes[i])
        return matching_pool
    
    @staticmethod
    def __crossover(matching_pool: Population, crossover_rate: float) -> Population:
        """
        Create a new population with the crossover of the matching pool chromosomes
        """
        new_population: Population = Population()
        for _ in range(len(matching_pool.chromosomes) // 2):
            chromosome_parent_1 = random.choice(matching_pool.chromosomes)
            chromosome_parent_2 = random.choice(matching_pool.chromosomes)
            if random.random() < crossover_rate:
                crossover_point = random.randint(1, len(chromosome_parent_1.genes) - 1)
                chromosome_child_1 = Chromosome()
                chromosome_child_2 = Chromosome()
                chromosome_child_1.genes = chromosome_parent_1.genes[:crossover_point] + chromosome_parent_2.genes[crossover_point:]
                chromosome_child_2.genes = chromosome_parent_2.genes[:crossover_point] + chromosome_parent_1.genes[crossover_point:]
                new_population.add_chromosome(chromosome_child_1)
                new_population.add_chromosome(chromosome_child_2)
            else:
                new_population.add_chromosome(chromosome_parent_1)
                new_population.add_chromosome(chromosome_parent_2)
        return new_population
    
    @staticmethod
    def __mutation(new_population: Population, mutation_rate: float, class_groups: List[ClassGroup], classrooms: List[Classroom], subjects: List[Subject], professors: List[Professor]) -> Population:
        """
        Mutate the genes of the new population with a random selection of the parameters
        """
        for chromosome in new_population.chromosomes:
            for gene in chromosome.genes:
                if random.random() < mutation_rate:
                    gene.generate_random_gene(class_groups, classrooms, subjects, professors)
        return new_population
    
    @staticmethod
    def __replace_population(population: Population, new_population: Population) -> Population:
        """
        Replace the population with the new population if the new population has better individuals
        """
        population.order_population_by_fitness()
        new_population.order_population_by_fitness(reverse=True)
        for i in range(len(new_population.chromosomes)):
            if new_population.chromosomes[i].elite_fitness > population.chromosomes[i].elite_fitness:
                population.chromosomes[i] = new_population.chromosomes[i]
        return population

    def run(self) -> MetricsEvaluator:
        """
        Run the genetic algorithm model and return the evaluation metrics
        """
        for generation in range(self.generations):
            self.metrics_evaluator.start_iteration()
            if generation > 0:
                matching_pool = self.__selection(self.population)
                new_population = self.__crossover(matching_pool, self.crossover_rate)
                new_population = self.__mutation(new_population, self.mutation_rate, self.class_groups, self.classrooms, self.subjects, self.professors)
                self.population = self.__replace_population(self.population, new_population)
            self.metrics_evaluator.finish_iteration(generation, self.population.best_chromosome, self.population.best_chromosome.avg_amount_of_conflicts, self.population.best_chromosome.elite_fitness)
            if self.population.best_chromosome.elite_fitness >= self.fitness_threshold:
                break
        self.metrics_evaluator.finish_evaluation()
        return self.metrics_evaluator

# 2 Experiment

## 2.1 Load Data

In [18]:
dataclasses_relationships = {
    "times": Time,
    "days": Day,
    "classes": ClassGroup,
    "rooms": Classroom,
    "subjects": Subject,
    "teachers": Professor,
}

In [19]:
timetabling_data = load_data(INPUT_FILE_PATH, dataclasses_relationships)

Loading classes data
5 of classes loaded
Loading rooms data
5 of rooms loaded
Loading subjects data
5 of subjects loaded
Loading teachers data
5 of teachers loaded
Loading times data
5 of times loaded
Loading days data
5 of days loaded
{'classes': [ClassGroup(id=1, students_amount=20), ClassGroup(id=2, students_amount=18), ClassGroup(id=3, students_amount=35), ClassGroup(id=4, students_amount=25), ClassGroup(id=5, students_amount=49)], 'rooms': [Classroom(id=1, capacity=60, available_days_ids=[1], available_time_ids=[1, 2, 3, 4, 5], given_availability=None), Classroom(id=2, capacity=30, available_days_ids=[1, 2], available_time_ids=[1, 2, 3, 4, 5], given_availability=None), Classroom(id=3, capacity=40, available_days_ids=[1, 2, 3], available_time_ids=[1, 2, 3, 4, 5], given_availability=None), Classroom(id=4, capacity=50, available_days_ids=[1, 5], available_time_ids=[1, 2, 3, 4, 5], given_availability=None), Classroom(id=5, capacity=45, available_days_ids=[6], available_time_ids=[1, 2,

## 2.2 Model Parameters

In [20]:
def calculate_quantity_of_classes_needed(subjects: List[Subject], classrooms: List[Classroom]) -> float:
    print(f"Calculating quantity of classes needed for {len(subjects)} subjects and {len(classrooms)} classrooms")
    quantity_of_classes: float = 0
    for subject in subjects:
        quantity_of_classes += subject.course_load // 1.5
    quantity_of_classes = quantity_of_classes * len(classrooms)
    print(f"Quantity of classes needed: {quantity_of_classes}")
    return quantity_of_classes

In [21]:
POPULATION_SIZE = 200
MUTATION_RATE = 0.4
CROSSOVER_RATE = 0.6
GENERATIONS = 500
FITNESS_THRESHOLD = 0.9

In [22]:
COURSE_LOAD_ONE_CLASS_THRESHOLD = 36

In [23]:
NUM_CLASSES = calculate_quantity_of_classes_needed(timetabling_data["subjects"], timetabling_data["rooms"])

Calculating quantity of classes needed for 5 subjects and 5 classrooms
Quantity of classes needed: 760.0


## 2.3 Genetic Selection

In [24]:
metrics_evaluator: MetricsEvaluator = GeneticAlgorithm(POPULATION_SIZE, MUTATION_RATE, CROSSOVER_RATE, GENERATIONS, FITNESS_THRESHOLD, COURSE_LOAD_ONE_CLASS_THRESHOLD, timetabling_data["classes"], timetabling_data["rooms"], timetabling_data["subjects"], timetabling_data["teachers"]).run()

Creating MetricsEvaluator of Model: Genetic Algorithm, starting evaluation
Generating population of size 100 with 5 class groups, 5 classrooms, 5 subjects and 5 professors and a course load threshold of 36 hours
Population generated with 100 chromosomes
Starting iteration 1
Iteration 0 - Conflicts: 48.235 - Elite Fitness: 0 - Time Elapsed: 1.997166633605957
Iteration 0 finished
Iteration 1 - Conflicts: 48.285 - Elite Fitness: 0 - Time Elapsed: 7.078922748565674
Iteration 1 finished
Iteration 2 - Conflicts: 48.26 - Elite Fitness: 0 - Time Elapsed: 12.10961627960205
Iteration 2 finished
Iteration 3 - Conflicts: 48.285 - Elite Fitness: 0 - Time Elapsed: 17.1962149143219
Iteration 3 finished
Iteration 4 - Conflicts: 48.26 - Elite Fitness: 0 - Time Elapsed: 22.356484174728394
Iteration 4 finished
Iteration 5 - Conflicts: 48.26 - Elite Fitness: 0 - Time Elapsed: 27.368692874908447
Iteration 5 finished
Iteration 6 - Conflicts: 48.21 - Elite Fitness: 0 - Time Elapsed: 32.60499620437622
Iterati

## 2.4 Results

In [25]:
metrics_evaluator

MetricsEvaluator(Genetic Algorithm)

In [26]:
metrics_evaluator.metrics.best_iteration.chromosome